# <div style="background: linear-gradient(135deg, #6e8efb, #a777e3);padding: 30px;border-radius: 15px;box-shadow: 0 4px 15px rgba(0,0,0,0.2);text-align: center;color: white;font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;"><h1 style="margin: 0; font-size: 40px; border: none; color: white;">Анализ личности клиента</h1><p style="margin: 10px 0 0; font-size: 18px; opacity: 0.9; text-align:center">Анализ данных, Кластеризация и создание моделей (+ Интерактивный Дашбоард)</p>
</div>

<div style="
    background-color: #f8f9fa;
    border-left: 8px solid #6e8efb;
    padding: 15px;
    border-radius: 5px;
    font-family: sans-serif;">
    <details>
        <summary style="font-weight: bold; font-size: 20px; cursor: pointer;">Данный файл является сводкой нескольких разных решений и файлов которые можно посмотреть тут</summary>
        <ul style="margin-top: 10px; line-height: 1.8;">
            <li><a href="./notebooks/Elements.ipynb" style="color: #6e8efb; text-decoration: none; font-weight: bold;">Шаблоны элементов для красивого и быстрого оформления markdown</a></li>
            <li><a href="./notebooks/eda.ipynb" style="color: #6e8efb; text-decoration: none; font-weight: bold;">1. EDA, Первончальный анализ данных</a></li>
            <li><a href="./notebooks/clustering.ipynb" style="color: #6e8efb; text-decoration: none; font-weight: bold;">2. Кластеризация и первоначальная обработка данных</a></li>
        </ul>
    </details>
</div>

<div style="padding: 15px; border-radius: 10px; background-color: #e3f2fd; border-left: 6px solid #2196f3; color: #0d47a1;">
    <strong>ℹ️ Описание:</strong><p>Анализ личности клиента помогает предприятию модифицировать продукт в зависимости от  целевых сегментов. Например, вместо того чтобы тратить деньги на маркетинг нового продукта для каждого клиента в базе данных, компания может проанализировать, какой сегмент наиболее склонен к покупке, и направить маркетинговые усилия именно на эту конкретную группу</p>
    <p>Данное иследование строиться как раз для того чтобы определить к какой группе относится тот или иной человек</p>
</div>

# <div style="padding: 15px; border-radius: 10px; background-color: #e3f2fd; border-left: 6px solid #21c2f3; color: #0d47a1;font-size:25px"><strong>Импорт беблиотек</strong></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import integrate, optimize
from sklearn import preprocessing, model_selection, metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from IPython.display import display
%matplotlib inline

# <div style="padding: 15px; border-radius: 10px; background-color: #e3f2fd; border-left: 6px solid #21c2f3; color: #0d47a1;font-size:25px"><strong>Импорт данных</strong></div>

In [3]:
df = pd.read_csv('./data/marketing_campaign.csv',sep='\t')

### <div style="padding: 15px; border-radius: 10px; background-color: #fff3e0; border-left: 6px solid #ff9800; color: #e65100;font-size:20px"><strong>⚠️ Важно:</strong> Перед запуском этой ячейки убедитесь, что в папке <code>/data</code> лежит файл <code>marketing_campaign.csv</code>.</div>

<div style="background:linear-gradient(135deg,#e3f2fd 0%,#f0e6ff 100%);border-left:6px solid #6e8efb;border-radius:12px;padding:20px 25px;box-shadow:0 4px 18px rgba(110,142,251,0.13);margin-bottom:18px;">
<span style="font-size:20px;font-weight:800;color:#0d47a1;letter-spacing:1px;">📋 Содержание (Признаки)</span>
<p style="color:#37474f;margin:8px 0 0;font-size:14px;">Описание всех признаков датасета, разбитых по группам.</p>
</div>

---

<div style="background:linear-gradient(120deg,#e3f2fd 60%,#e8eaff 100%);border-left:6px solid #2196f3;border-radius:10px;padding:16px 22px;box-shadow:0 2px 10px rgba(33,150,243,0.10);margin-bottom:16px;">
<strong style="font-size:17px;color:#0d47a1;">👤 Люди (People)</strong>

| Признак | Описание |
|---|---|
| <strong>ID</strong> | Уникальный идентификатор клиента |
| <strong>Year_Birth</strong> | Год рождения клиента |
| <strong>Education</strong> | Уровень образования |
| <strong>Marital_Status</strong> | Семейное положение |
| <strong>Income</strong> | Годовой доход домохозяйства клиента |
| <strong>Kidhome</strong> | Количество детей в семье |
| <strong>Teenhome</strong> | Количество подростков в семье |
| <strong>Dt_Customer</strong> | Дата регистрации клиента в компании |
| <strong>Recency</strong> | Количество дней с момента последней покупки |
| <strong>Complain</strong> | `1` — если клиент подавал жалобу в последние 2 года, иначе `0` |
</div>

<div style="background:linear-gradient(120deg,#e8f5e9 60%,#e8eaff 100%);border-left:6px solid #43a047;border-radius:10px;padding:16px 22px;box-shadow:0 2px 10px rgba(67,160,71,0.10);margin-bottom:16px;">
<strong style="font-size:17px;color:#1b5e20;">🛒 Продукты (Products)</strong>

| Признак | Описание |
|---|---|
| <strong>MntWines</strong> | Сумма, потраченная на вино за последние 2 года |
| <strong>MntFruits</strong> | Сумма, потраченная на фрукты за последние 2 года |
| <strong>MntMeatProducts</strong> | Сумма, потраченная на мясные продукты за последние 2 года |
| <strong>MntFishProducts</strong> | Сумма, потраченная на рыбу за последние 2 года |
| <strong>MntSweetProducts</strong> | Сумма, потраченная на сладости за последние 2 года |
| <strong>MntGoldProds</strong> | Сумма, потраченная на изделия из золота за последние 2 года |
</div>

<div style="background:linear-gradient(120deg,#fff8e1 60%,#ffe0e0 100%);border-left:6px solid #ffa000;border-radius:10px;padding:16px 22px;box-shadow:0 2px 10px rgba(255,160,0,0.10);margin-bottom:16px;">
<strong style="font-size:17px;color:#e65100;">📣 Продвижение (Promotion)</strong>

| Признак | Описание |
|---|---|
| <strong>NumDealsPurchases</strong> | Количество покупок, совершённых со скидкой |
| <strong>AcceptedCmp1</strong> | `1` — принял предложение в **1-й** кампании, иначе `0` |
| <strong>AcceptedCmp2</strong> | `1` — принял предложение во **2-й** кампании, иначе `0` |
| <strong>AcceptedCmp3</strong> | `1` — принял предложение в **3-й** кампании, иначе `0` |
| <strong>AcceptedCmp4</strong> | `1` — принял предложение в **4-й** кампании, иначе `0` |
| <strong>AcceptedCmp5</strong> | `1` — принял предложение в **5-й** кампании, иначе `0` |
| <strong>Response</strong> | `1` — принял предложение в **последней** кампании, иначе `0` |
</div>

<div style="background:linear-gradient(120deg,#fce4ec 60%,#e8eaff 100%);border-left:6px solid #e91e63;border-radius:10px;padding:16px 22px;box-shadow:0 2px 10px rgba(233,30,99,0.10);margin-bottom:16px;">
<strong style="font-size:17px;color:#880e4f;">🏪 Каналы продаж (Place)</strong>

| Признак | Описание |
|---|---|
| <strong>NumWebPurchases</strong> | Количество покупок через сайт компании |
| <strong>NumCatalogPurchases</strong> | Количество покупок по каталогу |
| <strong>NumStorePurchases</strong> | Количество покупок непосредственно в магазинах |
| <strong>NumWebVisitsMonth</strong> | Количество посещений сайта за последний месяц |
</div>